In [1]:
import xml.etree.ElementTree as ET 
import pandas as pd
import codecs
import os
import zipfile
import math
import numpy as np
import glob
import functools
import operator
from joblib import Parallel, delayed

In [2]:
#find all files of the specific type in the folder
def filebrowser(ext=""):
    return [f for f in glob.glob(f"*{ext}")]

In [5]:
os.chdir("C:\\Users\\fjying\\Desktop\\RA_Columbia\\patentparsing\\" + "input_2012")

In [6]:
data_list = os.listdir("C:\\Users\\fjying\\Desktop\\RA_Columbia\\patentparsing\\" + "input_2012")

In [7]:
#07-09
data_list = filebrowser('zip')
data_list

['ad20121231-01.zip',
 'ad20121231-02.zip',
 'ad20121231-03.zip',
 'ad20121231-04.zip',
 'ad20121231-05.zip',
 'ad20121231-06.zip',
 'ad20121231-07.zip',
 'ad20121231-08.zip',
 'ad20121231-09.zip',
 'ad20121231-10.zip',
 'ad20121231-11.zip',
 'ad20121231-12.zip']

In [ ]:
def parse_xml(pa):
    # no meaningful patent transfer: jump
    if pa.find('assignment-record/conveyance-text').text != 'ASSIGNMENT OF ASSIGNORS INTEREST (SEE DOCUMENT FOR DETAILS).':
        return

    patent_ids = []
    for patent_property in pa.findall('patent-properties/patent-property'):  
        for documentid in patent_property.findall('document-id'):
            doc_number = documentid.find('doc-number').text
            if len(doc_number) == 7:
                patent_ids.append(doc_number)
    #no patent id: jump
    if len(patent_ids) == 0:
        return
    totaldata = pd.DataFrame()
    totaldata['patent_ids'] = patent_ids
    totaldata['last_update_date'] = pa.find('assignment-record/last-update-date/date').text
    totaldata['recorded_date'] = pa.find('assignment-record/recorded-date/date').text
    totaldata['reel-no'] = pa.find('assignment-record/reel-no').text
    totaldata['frame-no'] = pa.find('assignment-record/frame-no').text

    #Only one assignor
    #No address for assignor in the record
    totaldata['assignor'] = pa.find('patent-assignors/patent-assignor/name').text

    #multiple assignees: by default 3            
    assignee_count = 0
    for assignee in pa.findall('patent-assignees/patent-assignee'):
        for address in assignee:
            totaldata[address.tag + '_' + str(assignee_count)] = address.text
        assignee_count+=1      
    totaldata['total_number_assignees'] = assignee_count
    return totaldata

In [ ]:
total =pd.DataFrame()
for i in data_list:
    zf  = zipfile.ZipFile(i, 'r')
    xml_names = zf.namelist()
    xmls = [zf.open(name) for name in xml_names]
    tree = ET.parse(xmls[0])
    root = tree.getroot()
    pas = root.findall('patent-assignments/patent-assignment')
    with Parallel(n_jobs=-1, verbose=1, prefer="threads") as parallel:
        results = parallel(delayed(parse_xml)(pa) for pa in pas)
    temp = pd.concat(results, ignore_index = False)
    total = total.append(temp)

In [ ]:
total.to_csv("2015.csv", index = False)

In [ ]:
with Parallel(n_jobs=-1, verbose=1, prefer="threads") as parallel:
    results = parallel(delayed(parse_xml)(pa) for pa in pas)

In [ ]:
total = pd.concat(results, ignore_index = False)

In [8]:
def add_recordeddate(pa):
    # no meaningful patent transfer: jump
    if pa.find('assignment-record/conveyance-text').text != 'ASSIGNMENT OF ASSIGNORS INTEREST (SEE DOCUMENT FOR DETAILS).':
        return  
    
    patent_count = 0
    recorded_dates = []
    for patent_property in pa.findall('patent-properties/patent-property'):  
        for documentid in patent_property.findall('document-id'):
            doc_number = documentid.find('doc-number').text
            if len(doc_number) == 7:
                patent_count += 1
    #no patent id: jump
    if patent_count == 0:
        return  

    recorded_dates.append(pa.find('assignment-record/recorded-date/date').text)
    recorded_dates = recorded_dates* patent_count
    return recorded_dates

In [9]:
zf  = zipfile.ZipFile('ad20121231-10.zip', 'r')
xml_names = zf.namelist()
xmls = [zf.open(name) for name in xml_names]
tree = ET.parse(xmls[0])
root = tree.getroot()
pas = root.findall('patent-assignments/patent-assignment')

In [ ]:
total = []
for i in data_list:
    zf  = zipfile.ZipFile(i, 'r')
    xml_names = zf.namelist()
    xmls = [zf.open(name) for name in xml_names]
    tree = ET.parse(xmls[0])
    root = tree.getroot()
    pas = root.findall('patent-assignments/patent-assignment')
    with Parallel(n_jobs=-1, verbose=1, prefer="threads") as parallel:
        results = parallel(delayed(add_recordeddate)(pa) for pa in pas)
        results = [i for i in results if i]
        total.extend(results)

In [10]:
total = []
with Parallel(n_jobs=-1, verbose=1, prefer="threads") as parallel:
    results = parallel(delayed(add_recordeddate)(pa) for pa in pas)
    #drop None objects
    results = [i for i in results if i]
    total.extend(results)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 2442 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 3192 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 4042 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 4992 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 6042 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 7192 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 8442 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done 9792 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 11242 tasks  

In [11]:
recorded_dates = functools.reduce(operator.iconcat, total, [])
recorded_dates

['20090315',
 '20090520',
 '20090520',
 '20090520',
 '20090520',
 '20090520',
 '20090520',
 '20090515',
 '20090520',
 '20090520',
 '20090520',
 '20090520',
 '20090520',
 '20090520',
 '20090520',
 '20090520',
 '20090520',
 '20090520',
 '20090520',
 '20090520',
 '20090520',
 '20090520',
 '20090520',
 '20090520',
 '20090508',
 '20090520',
 '20090520',
 '20090511',
 '20090511',
 '20090512',
 '20090520',
 '20090520',
 '20090511',
 '20090511',
 '20090515',
 '20090520',
 '20090520',
 '20090520',
 '20090519',
 '20090519',
 '20090515',
 '20090520',
 '20090520',
 '20090520',
 '20090520',
 '20090520',
 '20090520',
 '20090520',
 '20090520',
 '20090520',
 '20090520',
 '20090520',
 '20090515',
 '20090520',
 '20090520',
 '20090515',
 '20090515',
 '20090520',
 '20090520',
 '20090520',
 '20090520',
 '20090520',
 '20090519',
 '20090520',
 '20090520',
 '20090519',
 '20090519',
 '20090519',
 '20090520',
 '20090519',
 '20090520',
 '20090520',
 '20090520',
 '20090519',
 '20090520',
 '20090520',
 '20090520',

In [ ]:
#add record date

In [12]:
data = pd.read_csv('C:\\Users\\fjying\\Desktop\\RA_Columbia\\patentparsing\\output\\ad20121231-10.csv')

C:\Users\fjying\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (3,4,5,6,7,8,9,12,13,14,15,16,17,18,23,24,25,26,27,28,29,33,34,35,36,37,38,39,45,46,47,48,49,50,51,55,56,57,58,59,60,62,66,67,68,69,70,71) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
data['recorded_date'] = recorded_dates

In [14]:
data.to_csv("ad20121231-10.csv", index = False)

In [ ]:
data.columns